In [1]:
import os
import sys
import json

notebook_path = os.getcwd()
project_path = os.path.abspath(os.path.join(notebook_path, '..','..'))
print(f"Project path: {project_path}")
# Add the project path to sys.path
sys.path.append(project_path)

Project path: c:\Users\dalej\Documents\_Coding\DragonRegen


In [2]:
from src.AIGuardian.ActionsFrame.DataStructCreate import DataStructCreate

start_action = DataStructCreate()
prompt = """Create a set of tables for a simple online car insurance quoting engine."""
start_action.run(prompt)

create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <business_requirements> and create the table list and schema for the the request.\n        <business_requirements>\n        Create a set of tables for a simple online car insurance quoting engine.\n        </business_requirements>\n        Please provide the response in a JSON format like the <JSON_Template> below:\n        <JSON_Template>\n        {"table_name_1": {\n            "purpose": "short description of tables usage in the architecture",\n            "fields":{"column_name_1": "unique ID, short description of column",\n            , "column_name_2": "short description of column"\n            , "column_name_3": "short description of column"\n            }},\n        "table_name_2": {\n            "purpose": "short description of tables usage in the architecture",\n            "

In [3]:
print(start_action.output_params.keys())
print(json.dumps(start_action.output_params, indent=2)) 

dict_keys(['customers', 'vehicles', 'drivers', 'driving_history', 'coverage_options', 'quotes', 'quote_coverages', 'discount_types', 'quote_discounts'])
{
  "customers": {
    "purpose": "Stores information about customers who request car insurance quotes",
    "fields": {
      "customer_id": "unique ID, primary key for customer identification",
      "first_name": "customer's first name",
      "last_name": "customer's last name",
      "email": "customer's email address for communication",
      "phone_number": "customer's contact phone number",
      "date_of_birth": "customer's date of birth for age calculation",
      "address_line_1": "first line of customer's address",
      "address_line_2": "second line of customer's address (optional)",
      "city": "customer's city of residence",
      "state": "customer's state of residence",
      "zip_code": "customer's postal code",
      "created_at": "timestamp when customer record was created",
      "updated_at": "timestamp when cu

In [5]:
# Run the Scheme Refiner
from src.AIGuardian.ActionGenerator.SchemaRefiner import SchemaRefiner

test_schema_refiner = SchemaRefiner(input_params={}, parent_action=start_action)
print(test_schema_refiner)
test_schema_refiner.run(prompt)

In [8]:
test_schema_refiner.initialize()

test_schema_refiner.input_params

{'customers': {'purpose': 'Stores information about customers who request car insurance quotes',
  'fields': {'customer_id': 'unique ID, primary key for customer identification',
   'first_name': "customer's first name",
   'last_name': "customer's last name",
   'email': "customer's email address for communication",
   'phone_number': "customer's contact phone number",
   'date_of_birth': "customer's date of birth for age calculation",
   'address_line_1': "first line of customer's address",
   'address_line_2': "second line of customer's address (optional)",
   'city': "customer's city of residence",
   'state': "customer's state of residence",
   'zip_code': "customer's postal code",
   'created_at': 'timestamp when customer record was created',
   'updated_at': 'timestamp when customer record was last updated'}},
 'vehicles': {'purpose': 'Stores information about vehicles to be insured',
  'fields': {'vehicle_id': 'unique ID, primary key for vehicle identification',
   'customer_id

In [9]:
test_schema_refiner.run(prompt)

create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <column_information> to choose what type of column a column most likely is.\n<column_information>\nTable Purpose: Stores information about customers who request car insurance quotes\nColumn Info: "customer_id": "unique ID, primary key for customer identification"\n</column_information>\n\nChoose one column_type from this list <choices> below, structure as column_type: description of what qualifies a column as that type.\n<choices>\nColBasic: A back stop column for your basic data types with no other identifying attributes.\nStringBasic: This is a basic string column generator that can be used to generate a column of strings with a specified length and format. It is not intended for free text or complex string generation.\nCategorical: a column that represents a category or type, often 

In [11]:
print(test_schema_refiner.input_params.keys())
print(test_schema_refiner.output_params.keys())
print(json.dumps(test_schema_refiner.output_params, indent=2))

dict_keys(['customers', 'vehicles', 'drivers', 'driving_history', 'coverage_options', 'quotes', 'quote_coverages', 'discount_types', 'quote_discounts'])
dict_keys(['customers', 'vehicles', 'drivers', 'driving_history', 'coverage_options', 'quotes', 'quote_coverages', 'discount_types', 'quote_discounts'])
{
  "customers": {
    "purpose": "Stores information about customers who request car insurance quotes",
    "fields": [
      {
        "name": "customer_id",
        "type": "Integer",
        "nullable": false,
        "metadata": {
          "description": "unique ID, primary key for customer identification",
          "unique_fl": true,
          "default_value": null,
          "col_type": "Unique_Identifier"
        }
      },
      {
        "name": "first_name",
        "type": "String",
        "nullable": false,
        "metadata": {
          "description": "customer's first name",
          "unique_fl": false,
          "default_value": null,
          "col_type": "StringF

In [12]:
# export the schema to a JSON file
import json

output_file_path = os.path.join(notebook_path, 'output_schema.json')
with open(output_file_path, 'w') as output_file:
    json.dump(test_schema_refiner.output_params, output_file, indent=2)
print(f"Schema exported to {output_file_path}")

Schema exported to c:\Users\dalej\Documents\_Coding\DragonRegen\docs\AIGuardian\output_schema.json


## Try to create data

In [14]:
# load the schema from the JSON file
output_file_path = os.path.join(notebook_path, 'output_schema.json')
with open(output_file_path, 'r') as output_file:
    loaded_schema = json.load(output_file)

print(loaded_schema.keys())

dict_keys(['customers', 'vehicles', 'drivers', 'driving_history', 'coverage_options', 'quotes', 'quote_coverages', 'discount_types', 'quote_discounts'])


In [ ]:
from src.DataCreator.SchemaGenerators.SchemaSpark import SchemaSpark

# create spark and t-sql schemas with in there engines
sturct_col = SchemaSpark.generate_schema(loaded_schema)
print(loaded_schema.keys())
print(sturct_col.keys())
print('=='*20)
print(sturct_col['customers'])

ValueError: Invalid data type: Decimal. Valid data types are: Boolean, Byte, Short, Integer, Long, Float, Double, String, JSON, Binary, Date, Timestamp, Null, YearMonthInterval, DayTimeInterval

In [ ]:
# generate fake data for the tables
